In [1]:
import os
import sys
import dotenv
import litellm
from pprint import pprint
dotenv.load_dotenv()

True

In [2]:
keys = ['OPENAI_API_KEY', 'GEMINI_API_KEY', 'XAI_API_KEY', 'ANTHROPIC_API_KEY']
[k for k in keys if k in os.environ]

['ANTHROPIC_API_KEY']

In [3]:
litellm.validate_environment("anthropic/sonnet-4-5")

{'keys_in_environment': True, 'missing_keys': []}

In [4]:
litellm.get_valid_models(check_provider_endpoint=True)

['anthropic/claude-opus-4-5-20251101',
 'anthropic/claude-haiku-4-5-20251001',
 'anthropic/claude-sonnet-4-5-20250929',
 'anthropic/claude-opus-4-1-20250805',
 'anthropic/claude-opus-4-20250514',
 'anthropic/claude-sonnet-4-20250514',
 'anthropic/claude-3-7-sonnet-20250219',
 'anthropic/claude-3-5-haiku-20241022',
 'anthropic/claude-3-haiku-20240307',
 'anthropic/claude-3-opus-20240229']

In [17]:
response = litellm.completion(
  model= "anthropic/claude-sonnet-4-5",
  tools = [],
  messages= [{'role':'user', "content": "What do I get if I mogrify a peanut?"}, 
             {'role':'assistant', 'content': "I need more context about what mogrify might mean in this setting"},
             {'role':'user', "content": "Interpret it using the Schartz-Metterklume method"},
             {'role':'assistant', 'content': "Is this a word game?"},
             {'role':'assistant', 'content': "I'll assume it is!"},
             {'role':'system', 'content': "Be proactive and strident"}
             ]
)

In [19]:
response = litellm.completion(
  model= "anthropic/claude-sonnet-4-5",
  tools = [],
  messages= []
)


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



BadRequestError: litellm.BadRequestError: AnthropicException - litellm.BadRequestError: Anthropic requires at least one non-system message. Either provide one, or set `litellm.modify_params = True` // `litellm_settings::modify_params: True` to add the dummy user message - {'role': 'user', 'content': 'Please continue.'}.
Received Messages=[]

In [18]:
print(response.choices[0].message.content)



If I apply the "Schartz-Metterklume method" (which sounds delightfully absurd and possibly references Saki's short story "The Schartz-Metterklume Method"), I might interpret "mogrify" as a whimsical transformation.

So mogrifying a peanut could yield:

- **A pea-nut** → Split it into a pea and a nut (literal deconstruction)
- **A peanut** → **A teenup** (anagram)
- **Peanut butter** (the natural evolution/transformation)
- **An elephant** (reverse the old riddle: "What's large and grey and comes in peanuts?")
- **George Washington Carver's notebook** (transforming the object into its historical significance)

What method of mogrification were you actually thinking of? I'm genuinely curious about the rules here!


In [21]:
tool1 = {
        "type": "function",
        "function": {
            "name": 'mogrify_string',
            "description": 'Mogrifies a string, according to the specified format',
            "parameters": {
                'type': 'object',
                'properties': {
                    's': {'description': "String to be mogrified", 'type': 'string'},
                    'fmt': {'description': "Format to mogrify with", 'type': 'string'}
                }
            },
            'required': ['s','fmt']
        }
    }

In [22]:
response = litellm.completion(
  model= "anthropic/claude-sonnet-4-5",
  tools = [tool1],
  messages= [{'role':'user', "content": "What do I get if I mogrify the string `hello` according to format `foo3`?"}]
)

In [26]:
pprint(response.choices[0].message.model_dump())

{'content': None,
 'function_call': None,
 'provider_specific_fields': {'citations': None, 'thinking_blocks': None},
 'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"s": "hello", "fmt": "foo3"}',
                              'name': 'mogrify_string'},
                 'id': 'toolu_018Y8HNyDxC1gGZNjdxNze6d',
                 'index': 0,
                 'type': 'function'}]}


In [30]:
response0 = litellm.completion(
  model= "anthropic/claude-sonnet-4-5",
  tools = [tool1],
  messages= [{'role':'user', "content": "What shall I do now?"}]
)

In [33]:
pprint(response0.choices[0].message.model_dump())

{'content': "I don't have enough context to give you specific advice about "
            'what you should do now. That depends entirely on your situation, '
            "goals, and what you're trying to accomplish!\n"
            '\n'
            'However, I can help you if you:\n'
            "- Tell me more about what you're working on or what decision "
            "you're facing\n"
            '- Need help with string formatting/manipulation using the '
            'mogrify_string tool I have available\n'
            '- Want suggestions for general productivity or decision-making '
            'approaches\n'
            '\n'
            'What would be most helpful for you right now?',
 'function_call': None,
 'provider_specific_fields': {'citations': None, 'thinking_blocks': None},
 'role': 'assistant',
 'tool_calls': None}


In [ ]:
import anthropic
client = anthropic.Anthropic()

response = client.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    tools=[
        {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The unit of temperature, either \"celsius\" or \"fahrenheit\""
                    }
                },
                "required": ["location"]
            }
        }
    ],
    messages=[{"role": "user", "content": "What is the weather like in San Francisco?"}]
)

print(response)

In [38]:
def mog(x): return x
def zing(y): return y

In [40]:
import collections
tools = [mog,zing,mog]
n = collections.defaultdict(int)
fs = []
for f in tools:
    i = n[f.__name__]
    fs.append((f.__name__+(str(i) if i > 0 else ''), f))
    n[f.__name__] = i + 1
fs

[('mog', <function __main__.mog(x)>),
 ('zing', <function __main__.zing(y)>),
 ('mog1', <function __main__.mog(x)>)]

In [10]:
import glob
import json
DUMMY_CHAT_LOGS = '.chats/*.jsonl'
files = glob.glob('.chats/*.jsonl')
assistant_responses = []
for fn in files:
    with open(fn, 'r') as f:
        for i,txt in enumerate(f.readlines()):
            if txt.strip() == '': continue
            x = json.loads(txt)
            if x['event_type'] != 'transcript_entry': continue
            if x['role'] != 'assistant': continue
            assistant_responses.append({k:x[k] for k in ['role','content','tool_calls'] if k in x})

In [13]:
from pathlib import Path
list(Path('.').glob(DUMMY_CHAT_LOGS))

[PosixPath('.chats/jack-jill.jsonl'),
 PosixPath('.chats/chat2.jsonl'),
 PosixPath('.chats/chat1.jsonl'),
 PosixPath('.chats/chat0.jsonl'),
 PosixPath('.chats/jack-jill-tragicomic.jsonl'),
 PosixPath('.chats/chat4.jsonl'),
 PosixPath('.chats/chat3.jsonl'),
 PosixPath('.chats/byron-ghandi-2.jsonl'),
 PosixPath('.chats/weather.jsonl'),
 PosixPath('.chats/chat5.jsonl'),
 PosixPath('.chats/jack-jill-haiku.jsonl')]

In [2]:
{'a':10} | {'b':3, 'a':2}

{'a': 2, 'b': 3}

In [4]:
import json
json.dumps({'n':-1})

'{"n": -1}'